                                                                      **Iteration File** 

***Importing libraries and algorithm for Integrals***

In [1]:
import numpy as np
import time
import scf
import sympy as sp

In [2]:
r,r1,r2,zeta=sp.symbols("r,r1,r2,zeta")
n=sp.symbols("n", integer=True)

***The iteration code***

In [3]:
verbose=True 
MAXITER=50   #assigning the maximum number of iterations that should be carried out
E_convergence = 1.0e-7  #limit for energy convergence
def scf_hf_calc(basis,Z, charge):
    print("")
    print("---------------------------------------------------------------Intialization of SCF Calculation-------------------------------------------------------------------")
    print("-----------------------------------------------------------------Caclulating the Base Matrices--------------------------------------------------------------------")
    N = Z-charge
    #calling the fixed matrices
    start=time.time()
    #the overlap matrix
    s_mat=scf.S_mat(basis)

    #the core hamiltonian matrix
    h_core_mat=scf.H_core_mat(basis,Z)

    #Orbital Energies and Orbital Coefficient
    OrbE, Coef= scf.sec_eqn(h_core_mat,s_mat)

    #density matrix
    p_mat=scf.P_mat(Coef,N)


    
    scf.print_scf_data(OrbE, s_mat, h_core_mat, Coef, p_mat, True)
    F=h_core_mat
    #total energy calculation
    E=scf.E_total(p_mat,h_core_mat, F) 
    delta_E=1
    oldE= E
    print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------')
    stop=time.time()
    scf.print_total_E(E, start, stop, delta_E, False)

    print('------------------------------------------------------------Evaluation of Two Electron Integrals (takes time)-------------------------------------------------------')
    #the two electron repulsion 4D array
    r_mat_4D=scf.R_mat_4D(basis)


    ITER=0
    Wavefunctions=[]
    Densities=[]
    OrbitalEnergies=[]
    while ((delta_E>E_convergence) and (ITER<MAXITER)):
        print('-------------------------------------------------------------------------SCF ITERATION {}------------------------------------------------------------------------'.format(ITER+1))
        N=Z-charge
        start=time.time()
    
        #g matrix
        g_mat=scf.G_mat(p_mat, r_mat_4D)

        #fock matrix
        F=scf.F_mat(h_core_mat, g_mat)
        
        #secular equation
        OrbE, Coef= scf.sec_eqn(F,s_mat)

        #density matrix
        p_mat=scf.P_mat(Coef,N)

        #total energy calculation
        E=scf.E_total(p_mat,h_core_mat, F)
        delta_E=np.abs(E-oldE)
        oldE= E
        stop=time.time()
        scf.print_total_E(E, start, stop, delta_E, False)

        #storing wavefunctions for each iteration
        orbwfn=[]
        density=[]
        orbe=[]
        for i in range(Coef.shape[0]):
            psi=0
            for j, f in enumerate(basis):
                psi += Coef[j][i]*basis[j]
                psi_sq=psi*psi*4*np.pi*r*r
            orbwfn.append(psi)
            density.append(psi_sq)
            orbe.append(OrbE[i])
        g_es=tuple(orbwfn)
        d_es=tuple(density)
        e_es=tuple(orbe)
        Wavefunctions.append(g_es)
        Densities.append(d_es)
        OrbitalEnergies.append(e_es)
        ITER+=1

    return E, Coef, OrbE, Wavefunctions, OrbitalEnergies, Densities

***RHF SCF for Helium Atom*** 

In [4]:
def element1():
    #Helium Atom   He  1s2 
    Z=2
    charge=0
    start=time.time()
    reference=-2.8616726

    STO1=scf.STO(zeta=1.45363, n=1)      #--- 1ST BASIS FUNCTION
    STO2=scf.STO(zeta=2.91093, n=1)      #--- 2ND BASIS FUNCTION

    basis=[STO1, STO2]

    calculated, Coef, OrbE, Wavefunctions, OrbitalEnergies, Densities = scf_hf_calc(basis, Z, charge)
    
    stop=time.time()
    print('Total Elapsed Time: {:.2f} s'.format(stop-start))

    return calculated, Coef, OrbE, Wavefunctions, OrbitalEnergies, Densities

***RHF SCF for Berrylium Atom***

In [5]:
def element2():
    #Berrylium Atom  Be 1s22s2
    Z=4
    charge=0
    start=time.time()
    reference=-14.572369

    STO1=scf.STO(zeta=5.59108, n=1)
    STO2=scf.STO(zeta=3.35538, n=1)
    STO3=scf.STO(zeta=1.01122, n=2)
    STO4=scf.STO(zeta=0.61000, n=2)

    basis=[STO1, STO2, STO3, STO4]

    calculated, Coef, OrbE, Wavefunctions, OrbitalEnergies, Densities = scf_hf_calc(basis, Z, charge)
    
    stop=time.time()
    print('Total Elapsed Time: {:.2f} s'.format(stop-start))

    return calculated, Coef, OrbE, Wavefunctions, OrbitalEnergies, Densities

***Isoelectronic species for closed shell atoms***

***RHF SCF for $Be^{2+}$ ion***

In [6]:
def ion1():
    #Berrylium Atom  Be 1s22s2
    Z=4
    charge=2
    start=time.time()
    reference=-13.6112970

    STO1=scf.STO(zeta=3.43071, n=1)
    STO2=scf.STO(zeta=5.63150, n=1)
    STO3=scf.STO(zeta=7.35143 , n=1)

    basis=[STO1, STO2, STO3]


    calculated, Coef, OrbE, Wavefunctions, OrbitalEnergies, Densities = scf_hf_calc(basis, Z, charge)
    
    stop=time.time()
    print('Total Elapsed Time: {:.2f} s'.format(stop-start))

    return calculated, Coef, OrbE, Wavefunctions, OrbitalEnergies, Densities

***Ionisation Energy $Be^{2+}$***

In [7]:
ie = scf.ionisation(calculated2,calculated_i)

NameError: name 'calculated2' is not defined